In [6]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

batch_size = 64 # define your batch size as 64

train_dataset = datasets.MNIST(root='/data/', train = True, transform = transforms.ToTensor(), download = True) # load MNIST dataset
test_dataset = datasets.MNIST(root='/data/', train = False, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True) # load your data
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False) # load your data

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5) # 2d convolutional layer, 1 channel in, 10 channels out, kernel size of 5
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5) # 2d convolutional layer, 10 channels in, 20 channels out, kernel size of 5
        self.mp = nn.MaxPool2d(2) # Max pool function to reduce parameters
        self.fc = nn.Linear(320, 10) # fully connected layer
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x))) # take x and plug in to convolutional layer, max pool, then run activation function
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten into one layer
        x = self.fc(x)
        return F.log_softmax(x, dim = 1)
    
model = CNN()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # Stochastic gradient descent, learning rate of 0.01, momentum(learning rate adjustment) of 0.5

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): # get batch index and (data, target) tuple for each in the train data
        data, target = Variable(data), Variable(target) # wrap into pytorch variables
        optimizer.zero_grad() # zero the optimizer
        output = model(data)
        loss = F.nll_loss(output, target) # calculate the loss
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch, batch_idx*len(data), len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target) # wrap into pytorch variables
        output = model(data)
        
        test_loss += F.nll_loss(output, target, size_average = False).item() # add the current loss to the total test loss
        
        pred = output.data.max(1, keepdim = True)[1] # get the pred
        correct += pred.eq(target.data.view_as(pred)).cuda().sum() # check to see of the prediction is equal to the test value
        
    test_loss /= len(test_loader.dataset)
    print('\nTest Set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%) \n'.format(test_loss, correct, len(test_loader.dataset),
                                                                                 100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.286716
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.283609
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.255632
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.241706
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.223526
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.168379
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.083968
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.015280
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.899505
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.685139
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.429190
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.217673
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.141152
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.798085
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.762229
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.690877
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.668245
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.556084
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.584830
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.558350
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.149276
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.082457
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.176584
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.185138
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.125170
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.229020
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.337032
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.185643
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.117242
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.343668
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.137839
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.142145
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.141300
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.097927
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.200758
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.072059
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.142117
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.082110
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.178338
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.127973


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.184026
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.190416
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.071596
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.107152
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.070558
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.068661
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.115669
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.220222
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.053333
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.107243
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.185458
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.051053
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.041025
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.041638
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.046321
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.072884
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.133985
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.223422
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.070742
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.128972


Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.049468
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.035929
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.176364
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.051185
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.051165
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.124762
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.027301
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.099497
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.217415
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.193478
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.028720
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.075683
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.030649
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.120645
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.138246
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.076965
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.098855
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.114538
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.083152
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.189986


Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.034584
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.060639
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.098648
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.195440
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.029616
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.140808
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.032841
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.059835

Test Set: Average Loss: 0.0609, Accuracy: 9801/10000 (98%) 

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.027347
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.144792
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.004619
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.304163
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.024913
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.109253
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.012765
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.037873
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.121097
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.067699
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.100068
Train Epoch:

Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.139855
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.027543
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.009448
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.044179
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.019139
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.062736
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.011794
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.106747
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.007252
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.044475
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.066455
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.051706
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.025398
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.078675
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.019179
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.064642
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.049070
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.031677
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.112480
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.086496
